# Linear regression

Model trying to predict value of a variable($y_i$) using a linear relationship with another variable($x$) or variables<br>
$$\hat y_i = b_0 + b_1x$$
where $\hat y_i$ is the predicted value<br>

residual: absolute of (predicted value minus the actual value) $|\hat y_i - y_i|$<br><br>
We're trying to minimise sum of the residuals $\min(\sum_i|\hat y_i - y_i|)$<br>
We could also attribute a squared residual penalty for a model and try to minimise that $\min(\sum_i(\hat y_i - y_i)^2)$<br>

simple linear regression: $y_i = b_0 + b_1x$<br>
multiple linear regression: $y_i = b_0 + b_1x_0 + b_2x_1 + ... b_nx_{n-1}$<br>

Assumptions for modelling with linear regression:
 - linearity: (y should be linearly distributed w.r.t x)
 - independence: there is no relationship between y and residuals
 - normality: the residuals plot w.r.t. x must be normally distributed
 - homoscedasticity: the residual plot w.r.t. x must have equal variances across different sections

### Evaluating linear regression

#### Mean absolute error
$\dfrac{\sum_i^n|\hat y_i - y_i|}{n}$<br>

#### Mean squared error
$\dfrac{\sum_i^n(\hat y_i - y_i)^2}{n}$<br>

#### Root Mean squared error
$\sqrt{\dfrac{\sum_i^n(\hat y_i - y_i)^2}{n}}$<br>

#### $R^2$ - Coff. of determination
$R^2 = 1 - \dfrac{RSS}{TSS}$<br>
where RSS is the sum of squared residuals<br>
$RSS = \sum_{i = 1}^{n}(y_i - \hat y_i)^2$
and TSS is the total sum of squares<br>
$TSS = \sum_{i = 1}^{n}(y_i - \bar y_i)^2$, where $\bar y_i$ is the mean of $y_i$s